# High-Performance Optimization with HiGHS
## Linear Programming (LP) and Mixed-Integer Linear Programming (MILP)

This notebook demonstrates two real-world optimization problems:
1. **LP Problem**: Diet Optimization (continuous variables)
2. **MILP Problem**: Facility Location (mixed integer/continuous variables)


NOTE: MOST OF THIS CODE WAS GENERATED BY CLAUDE CODE - NOV 2025 

# Top 5 Commercial MILP Solvers

### 1. Gurobi Optimizer One of the most powerful and widely-used commercial solvers, known for exceptional performance on large-scale problems. Offers parallel processing, excellent presolve techniques, and strong community support. Popular in industry and academia.

### 2. CPLEX (IBM ILOG CPLEX Optimization Studio) IBM's flagship solver with decades of development. Excellent for enterprise applications with robust integration capabilities, strong performance, and comprehensive documentation. Widely used in supply chain, logistics, and operations research.

### 3. FICO Xpress Optimizer A high-performance solver with strong modeling capabilities through its Mosel language. Known for good performance on both LP and MILP problems, with particular strength in handling difficult integer programming problems.

### 4. MOSEK Excels particularly at conic optimization but also offers strong MILP capabilities. Known for numerical stability and efficiency, especially on problems with special structure. Often used in finance and engineering applications.

### 5. COPT (Cardinal Optimizer) A relatively newer entrant from Cardinal Operations, showing impressive performance benchmarks and competitive pricing. Has been gaining traction particularly in Asia and among users seeking alternatives to established solvers.

# Top 5 Open Source MILP Solvers

### 1. HiGHS A modern, MIT-licensed solver developed at the University of Edinburgh. Offers surprisingly competitive performance with commercial solvers, particularly on LP problems, with rapidly improving MILP capabilities. Written in C++ with interfaces for multiple languages.

### 2. SCIP (Solving Constraint Integer Programs) One of the most comprehensive non-commercial solvers, distributed under Apache 2.0 license. Offers a flexible framework for branch-cut-and-price, with strong academic backing from Zuse Institute Berlin. Highly extensible for research.

### 3. GLPK (GNU Linear Programming Kit) The veteran of open source optimization, widely used for educational purposes and smaller applications. While not competitive with commercial solvers on large problems, it's reliable, well-documented, and has minimal dependencies.

### 4. CBC (COIN-OR Branch and Cut) Part of the COIN-OR project, this EPL-licensed solver is widely used in open source optimization stacks. Good performance for many practical problems and integrates well with other COIN-OR tools. Actively maintained.

### 5. SYMPHONY Also part of COIN-OR, designed for solving large-scale MILP problems in parallel computing environments. Good for customization and research applications where you need access to the solution process internals.
# Performance Considerations


### Commercial solvers (Gurobi, CPLEX, Xpress) generally offer superior performance on large, difficult problems and provide professional support, but come with licensing costs. Open source options like HiGHS and SCIP have closed the gap significantly in recent years and are excellent choices for many applications, especially if budget is a constraint or if you need to embed a solver in open source software. 


## OK! LET'S TEST & EXPLORE - using Open Source HiGHS for: 
##   (1) LP Problem: Diet Optimization (continuous variables)
##   (2) MILP Problem: Facility Location (mixed integer/continuous variables)


In [19]:
# Install HiGHS if not already installed
# Uncomment the line below if you need to install
!pip install highspy

import highspy
import numpy as np
import pandas as pd
import time
from IPython.display import display, Markdown

print(f"HiGHS version: {highspy.HIGHS_VERSION_MAJOR}.{highspy.HIGHS_VERSION_MINOR}.{highspy.HIGHS_VERSION_PATCH}")
print("Setup complete!")

HiGHS version: 1.12.0
Setup complete!


---
## Part 1: Linear Programming (LP) - Diet Optimization

### Problem Description
Find the minimum-cost combination of foods that meets daily nutritional requirements.

### Mathematical Formulation

**Decision Variables:**
- $x_1$ = servings of chicken breast
- $x_2$ = servings of brown rice
- $x_3$ = servings of broccoli
- $x_4$ = servings of milk

**Objective Function:**
$$\text{Minimize Cost} = 3.50x_1 + 0.80x_2 + 1.20x_3 + 1.00x_4$$

**Subject to Constraints:**
- Protein: $30x_1 + 5x_2 + 3x_3 + 8x_4 \geq 60$ grams
- Calories: $200x_1 + 220x_2 + 50x_3 + 150x_4 \geq 2000$ kcal
- Calcium: $20x_1 + 10x_2 + 50x_3 + 300x_4 \geq 1000$ mg
- Fiber: $0x_1 + 2x_2 + 3x_3 + 0x_4 \geq 25$ grams
- Non-negativity: $x_1, x_2, x_3, x_4 \geq 0$

In [20]:
def solve_diet_problem():
    """Solve the diet optimization LP problem."""
    
    # Create HiGHS model
    h = highspy.Highs()
    h.setOptionValue("log_to_console", False)  # Suppress solver output
    
    # Problem setup
    num_vars = 4
    
    # Objective coefficients (costs per serving)
    costs = [3.50, 0.80, 1.20, 1.00]
    
    # Variable bounds (all continuous, non-negative)
    lower_bounds = [0.0] * num_vars
    upper_bounds = [highspy.kHighsInf] * num_vars
    
    # Add variables to model
    for i in range(num_vars):
        h.addVar(lower_bounds[i], upper_bounds[i])
    
    # Set objective (minimize)
    h.changeColsCost(num_vars, np.arange(num_vars), np.array(costs))
    h.changeObjectiveSense(highspy.ObjSense.kMinimize)
    
    # Constraint matrix (nutrients per serving)
    # Rows: Protein, Calories, Calcium, Fiber
    # Cols: Chicken, Rice, Broccoli, Milk
    constraint_matrix = [
        [30, 5, 3, 8],      # Protein (g)
        [200, 220, 50, 150], # Calories (kcal)
        [20, 10, 50, 300],   # Calcium (mg)
        [0, 2, 3, 0]         # Fiber (g)
    ]
    
    # Right-hand side (minimum requirements)
    rhs = [60, 2000, 1000, 25]
    
    # Add constraints (all >=, so lower bound = rhs, upper bound = infinity)
    for i, row in enumerate(constraint_matrix):
        indices = list(range(num_vars))
        values = row
        h.addRow(rhs[i], highspy.kHighsInf, len(indices), 
                 np.array(indices), np.array(values))
    
    # Solve the LP
    h.run()
    
    # Get solution
    solution = h.getSolution()
    info = h.getInfo()
    
    return h, solution, info, constraint_matrix, rhs, costs

# Solve the problem
start_time = time.time()
h_lp, solution_lp, info_lp, constraint_matrix, rhs, costs = solve_diet_problem()
solve_time_lp = time.time() - start_time

print("=" * 70)
print("LP PROBLEM: DIET OPTIMIZATION")
print("=" * 70)
print(f"\nStatus: {h_lp.modelStatusToString(h_lp.getModelStatus())}")
print(f"Optimal Cost: ${info_lp.objective_function_value:.2f}")
print(f"Solution Time: {solve_time_lp:.4f} seconds\n")

LP PROBLEM: DIET OPTIMIZATION

Status: Optimal
Optimal Cost: $12.45
Solution Time: 0.0020 seconds



In [21]:
# Display optimal diet as a DataFrame
foods = ["Chicken Breast", "Brown Rice", "Broccoli", "Milk"]

diet_df = pd.DataFrame({
    'Food': foods,
    'Servings': [solution_lp.col_value[i] for i in range(4)],
    'Cost per Serving': costs,
    'Total Cost': [solution_lp.col_value[i] * costs[i] for i in range(4)]
})

print("\n📊 OPTIMAL DIET:")
display(diet_df.style.format({
    'Servings': '{:.3f}',
    'Cost per Serving': '${:.2f}',
    'Total Cost': '${:.2f}'
}))


📊 OPTIMAL DIET:


,Food,Servings,Cost per Serving,Total Cost
0,Chicken Breast,0.000,$3.50,$0.00
1,Brown Rice,6.513,$0.80,$5.21
2,Broccoli,3.992,$1.20,$4.79
3,Milk,2.451,$1.00,$2.45


In [22]:
# Verify nutritional constraints
nutrients = ["Protein (g)", "Calories (kcal)", "Calcium (mg)", "Fiber (g)"]
nutrient_totals = [
    sum(constraint_matrix[i][j] * solution_lp.col_value[j] for j in range(4))
    for i in range(4)
]

nutrition_df = pd.DataFrame({
    'Nutrient': nutrients,
    'Actual': nutrient_totals,
    'Required': rhs,
    'Surplus': [nutrient_totals[i] - rhs[i] for i in range(4)]
})

print("\n🥗 NUTRITIONAL CONTENT:")
display(nutrition_df.style.format({
    'Actual': '{:.1f}',
    'Required': '{:.1f}',
    'Surplus': '{:.1f}'
}))


🥗 NUTRITIONAL CONTENT:


,Nutrient,Actual,Required,Surplus
0,Protein (g),64.1,60.0,4.1
1,Calories (kcal),2000.0,2000.0,-0.0
2,Calcium (mg),1000.0,1000.0,0.0
3,Fiber (g),25.0,25.0,0.0


---
## Part 2: Mixed-Integer Linear Programming (MILP) - Facility Location

### Problem Description
A company has 3 potential warehouse locations and 4 customer zones. Each warehouse has a fixed opening cost and variable shipping costs. Decide which warehouses to open and how to serve customers to minimize total cost.

### Mathematical Formulation

**Decision Variables:**
- $y_j \in \{0,1\}$: binary variable, 1 if warehouse $j$ is opened, 0 otherwise
- $x_{ij} \geq 0$: continuous variable, units shipped from warehouse $j$ to customer $i$

**Objective Function:**
$$\text{Minimize Total Cost} = \sum_{j} F_j y_j + \sum_{i,j} c_{ij} x_{ij}$$

Where $F_j$ is the fixed cost of opening warehouse $j$, and $c_{ij}$ is the shipping cost.

**Subject to Constraints:**
1. **Demand satisfaction**: $\sum_j x_{ij} = d_i$ for each customer $i$
2. **Capacity constraints**: $\sum_i x_{ij} \leq C_j y_j$ for each warehouse $j$
3. **Logical constraints**: $x_{ij} \leq d_i y_j$ (can only ship if warehouse is open)
4. **Non-negativity and integrality**: $x_{ij} \geq 0$, $y_j \in \{0,1\}$

In [23]:
def solve_facility_location():
    """Solve the facility location MILP problem."""
    
    # Create HiGHS model
    h = highspy.Highs()
    h.setOptionValue("log_to_console", False)
    
    # Problem data
    num_warehouses = 3
    num_customers = 4
    
    # Fixed costs to open each warehouse
    fixed_costs = [10000, 8000, 12000]
    
    # Capacity of each warehouse
    capacities = [500, 400, 600]
    
    # Customer demands
    demands = [150, 200, 180, 120]
    
    # Shipping costs from warehouse j to customer i
    shipping_costs = [
        [8, 6, 10, 9],    # From warehouse 1
        [9, 12, 7, 8],    # From warehouse 2
        [14, 8, 11, 6]    # From warehouse 3
    ]
    
    # Total variables: 3 binary + 12 continuous = 15
    num_binary_vars = num_warehouses
    num_flow_vars = num_warehouses * num_customers
    total_vars = num_binary_vars + num_flow_vars
    
    # Add binary variables (y) for warehouse opening decisions
    for j in range(num_warehouses):
        h.addVar(0.0, 1.0)
        h.changeColIntegrality(j, highspy.HighsVarType.kInteger)
    
    # Add continuous variables (x) for shipment flows
    for j in range(num_warehouses):
        for i in range(num_customers):
            h.addVar(0.0, highspy.kHighsInf)
    
    # Set objective coefficients
    obj_coeffs = []
    obj_coeffs.extend(fixed_costs)
    for j in range(num_warehouses):
        for i in range(num_customers):
            obj_coeffs.append(shipping_costs[j][i])
    
    h.changeColsCost(total_vars, np.arange(total_vars), np.array(obj_coeffs))
    h.changeObjectiveSense(highspy.ObjSense.kMinimize)
    
    # Helper function
    def get_x_index(warehouse_j, customer_i):
        return num_binary_vars + warehouse_j * num_customers + customer_i
    
    # CONSTRAINT 1: Demand satisfaction
    for i in range(num_customers):
        indices = [get_x_index(j, i) for j in range(num_warehouses)]
        values = [1.0] * num_warehouses
        h.addRow(demands[i], demands[i], len(indices), 
                 np.array(indices), np.array(values))
    
    # CONSTRAINT 2: Capacity constraints
    for j in range(num_warehouses):
        indices = [get_x_index(j, i) for i in range(num_customers)]
        indices.append(j)
        values = [1.0] * num_customers + [-capacities[j]]
        h.addRow(-highspy.kHighsInf, 0.0, len(indices), 
                 np.array(indices), np.array(values))
    
    # CONSTRAINT 3: Logical constraints
    for j in range(num_warehouses):
        for i in range(num_customers):
            indices = [get_x_index(j, i), j]
            values = [1.0, -demands[i]]
            h.addRow(-highspy.kHighsInf, 0.0, len(indices), 
                     np.array(indices), np.array(values))
    
    # Solve the MILP
    h.run()
    
    solution = h.getSolution()
    info = h.getInfo()
    
    return (h, solution, info, num_warehouses, num_customers, 
            fixed_costs, capacities, demands, shipping_costs, get_x_index)

# Solve the problem
start_time = time.time()
result = solve_facility_location()
solve_time_milp = time.time() - start_time

(h_milp, solution_milp, info_milp, num_warehouses, num_customers, 
 fixed_costs, capacities, demands, shipping_costs, get_x_index) = result

print("=" * 70)
print("MILP PROBLEM: FACILITY LOCATION")
print("=" * 70)
print(f"\nStatus: {h_milp.modelStatusToString(h_milp.getModelStatus())}")
print(f"Optimal Total Cost: ${info_milp.objective_function_value:,.2f}")
print(f"Solution Time: {solve_time_milp:.4f} seconds\n")

MILP PROBLEM: FACILITY LOCATION

Status: Optimal
Optimal Total Cost: $22,620.00
Solution Time: 0.0158 seconds



In [24]:
# Display warehouse opening decisions
warehouse_data = []
total_fixed_cost = 0

for j in range(num_warehouses):
    is_open = solution_milp.col_value[j] > 0.5
    if is_open:
        total_fixed_cost += fixed_costs[j]
    
    warehouse_data.append({
        'Warehouse': f'Warehouse {j+1}',
        'Status': '✅ OPEN' if is_open else '❌ CLOSED',
        'Fixed Cost': f'${fixed_costs[j]:,}',
        'Capacity': capacities[j]
    })

warehouse_df = pd.DataFrame(warehouse_data)
print("\n🏭 WAREHOUSE OPENING DECISIONS:")
display(warehouse_df)
print(f"\nTotal Fixed Costs: ${total_fixed_cost:,}")


🏭 WAREHOUSE OPENING DECISIONS:


,Warehouse,Status,Fixed Cost,Capacity
0,Warehouse 1,✅ OPEN,"$10,000",500
1,Warehouse 2,✅ OPEN,"$8,000",400
2,Warehouse 3,❌ CLOSED,"$12,000",600



Total Fixed Costs: $18,000


In [25]:
# Display shipment plan
shipment_data = []
total_shipping_cost = 0

for j in range(num_warehouses):
    if solution_milp.col_value[j] > 0.5:  # Only show open warehouses
        for i in range(num_customers):
            x_idx = get_x_index(j, i)
            quantity = solution_milp.col_value[x_idx]
            if quantity > 0.01:  # Only show non-zero shipments
                cost = quantity * shipping_costs[j][i]
                total_shipping_cost += cost
                shipment_data.append({
                    'From': f'Warehouse {j+1}',
                    'To': f'Customer {i+1}',
                    'Units': quantity,
                    'Cost/Unit': shipping_costs[j][i],
                    'Total Cost': cost
                })

shipment_df = pd.DataFrame(shipment_data)
print("\n🚚 SHIPMENT PLAN:")
display(shipment_df.style.format({
    'Units': '{:.1f}',
    'Cost/Unit': '${:.0f}',
    'Total Cost': '${:.2f}'
}))

print(f"\nTotal Shipping Costs: ${total_shipping_cost:,.2f}")
print(f"Total Fixed Costs: ${total_fixed_cost:,}")
print(f"Grand Total: ${total_fixed_cost + total_shipping_cost:,.2f}")


🚚 SHIPMENT PLAN:


,From,To,Units,Cost/Unit,Total Cost
0,Warehouse 1,Customer 1,150.0,$8,$1200.00
1,Warehouse 1,Customer 2,200.0,$6,$1200.00
2,Warehouse 2,Customer 3,180.0,$7,$1260.00
3,Warehouse 2,Customer 4,120.0,$8,$960.00



Total Shipping Costs: $4,620.00
Total Fixed Costs: $18,000
Grand Total: $22,620.00


In [26]:
# Verify demand satisfaction
demand_check = []
for i in range(num_customers):
    total_received = sum(solution_milp.col_value[get_x_index(j, i)] 
                        for j in range(num_warehouses))
    demand_check.append({
        'Customer': f'Customer {i+1}',
        'Demand': demands[i],
        'Received': total_received,
        'Status': '✓ Satisfied' if abs(total_received - demands[i]) < 0.01 else '✗ Not Met'
    })

demand_df = pd.DataFrame(demand_check)
print("\n📦 DEMAND SATISFACTION CHECK:")
display(demand_df.style.format({
    'Demand': '{:.0f}',
    'Received': '{:.1f}'
}))


📦 DEMAND SATISFACTION CHECK:


,Customer,Demand,Received,Status
0,Customer 1,150,150.0,✓ Satisfied
1,Customer 2,200,200.0,✓ Satisfied
2,Customer 3,180,180.0,✓ Satisfied
3,Customer 4,120,120.0,✓ Satisfied


---
## Summary and Key Insights

### LP vs MILP Comparison

| Aspect | LP (Diet Problem) | MILP (Facility Location) |
|--------|------------------|---------------------------|
| **Variables** | All continuous | Mixed (binary + continuous) |
| **Solution Speed** | Very fast (milliseconds) | Slower (branch-and-bound) |
| **Solution Nature** | Fractional values allowed | Integer constraints enforced |
| **Algorithm** | Simplex/Interior Point | Branch-and-bound + cuts |
| **Complexity** | Polynomial time | NP-hard |

### When to Use Each

**Use LP when:**
- Decisions are naturally continuous (quantities, percentages, flows)
- Fractional solutions make sense
- Need fast solutions for large problems

**Use MILP when:**
- Decisions are binary (yes/no, on/off)
- Need integer quantities (people, machines, facilities)
- Fixed costs or logical constraints are involved
- Solution must represent discrete choices